In [2]:
import numpy as np 
import pandas as pd
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


In [3]:
data_dir = "/Users/nimamanaf/Desktop/kaggle/pd/data/amex-default-prediction/"
BATCH_SIZE = 2064

In [4]:
cat_cols = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
data_temp = pd.read_csv(data_dir+"train_data.csv", nrows=5)
num_cols = [col for col in data_temp.columns.to_list() if col not in cat_cols]

In [8]:
train_customers = pd.read_csv(data_dir+"train_data.csv", usecols=["customer_ID"], engine="pyarrow")
test_customers = pd.read_csv(data_dir+"test_data.csv", usecols=["customer_ID"], engine="pyarrow")

: 

In [56]:
train_customer_indices = train_customers.reset_index().set_index("customer_ID").groupby('customer_ID').apply(lambda x : x.to_numpy().reshape(-1, )).to_dict()
test_customer_indices = test_customers.reset_index().set_index("customer_ID").groupby('customer_ID').apply(lambda x : x.to_numpy().reshape(-1, )).to_dict()

In [297]:
train_data = pd.read_parquet(data_dir+"train_data.parquet")

'1.4.2'

In [295]:

cat_cols = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

class TrainCustomerData(Dataset):
    def __init__(self, customer_indices, data_dir=None, cat_cols=cat_cols):
        #self.customer_indices = customer_indices
        self.customer_ids = tuple(customer_indices)
        self.customer_indices = tuple(customer_indices.values())
        self.data_dir = data_dir + "train_data.csv"
        self.train_labels = pd.read_csv(data_dir+"train_labels.csv", engine="pyarrow").set_index("customer_ID")
        self.data_columns = pd.read_csv(data_dir+"train_data.csv", nrows=5).columns.to_list()
        self.cont_cols = [col for col in self.data_columns if col not in cat_cols + ["customer_ID", "S_2"]]

    def __len__(self):
        return len(self.customer_indices)

    def __getitem__(self, index):
        customer_data_indices = self.customer_indices[index]
        skiprows = range(1, customer_data_indices[0]+1)
        nrows = customer_data_indices[-1] - customer_data_indices[0] + 1
        customer_data = pd.read_csv(self.data_dir, skiprows=skiprows, nrows=nrows, header=0, engine="pyarrow")
        customer_id = customer_data.customer_ID.iloc[0]
        
        customer_data.drop(["customer_ID", "S_2"], axis=1, inplace=True)
        
        customer_cont_data = customer_data[self.cont_cols].fillna(0, axis=1)
        customer_cont_tensor_data = torch.as_tensor(customer_cont_data.values, dtype=torch.float32)
        
        customer_cat_data = customer_data[cat_cols].values
        
        customer_label = torch.as_tensor(self.train_labels.loc[customer_id].values, dtype=torch.float32)
        
        return customer_cont_tensor_data[-1, :], customer_label


class TestCustomerData(Dataset):
    def __init__(self, customer_indices, data_dir=None, cat_cols=cat_cols):
        #self.customer_indices = customer_indices
        self.customer_ids = tuple(customer_indices)
        self.customer_indices = tuple(customer_indices.values())
        self.data_dir = data_dir + "test_data.csv"
        self.data_columns = pd.read_csv(data_dir+"test_data.csv", nrows=5).columns.to_list()
        self.cont_cols = [col for col in self.data_columns if col not in cat_cols + ["customer_ID", "S_2"]]

    def __len__(self):
        return len(self.customer_indices)

    def __getitem__(self, index):
        customer_data_indices = self.customer_indices[index]
        skiprows = range(1, customer_data_indices[0]+1)
        nrows = customer_data_indices[-1] - customer_data_indices[0] + 1
        customer_data = pd.read_csv(self.data_dir, skiprows=skiprows, nrows=nrows, header=0)
        customer_id = customer_data.customer_ID.iloc[0]
        
        customer_data.drop(["customer_ID", "S_2"], axis=1, inplace=True)
        
        customer_cont_data = customer_data[self.cont_cols].fillna(0, axis=1)
        customer_cont_tensor_data = torch.as_tensor(customer_cont_data.values, dtype=torch.float32)
    
        customer_cat_data = customer_data[cat_cols].values
        
        customer_label = torch.as_tensor(self.train_labels.loc[customer_id].values, dtype=torch.float32)
        
        return customer_cont_tensor_data[-1, :], customer_label



In [296]:
labels = pd.read_csv(data_dir+"train_labels.csv")
train_dataset = TrainCustomerData(train_customer_indices, data_dir=data_dir)
test_dataset = TestCustomerData(test_customer_indices, data_dir=data_dir)


ValueError: Unknown engine: pyarrow (valid options are dict_keys(['c', 'python', 'python-fwf']))

In [285]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)


In [280]:
class Model(nn.Module):
    def __init__(self, input_dim):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(in_features=input_dim, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=1)
        

    def forward(self, x):
        x = F.selu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))

        return x
        
    


In [290]:
model = Model(input_dim=177)
optimizer = torch.optim.Adam(model.parameters(),)
criterion = torch.nn.BCELoss()

for epoch in range(1): 
    for cont_feat, clabel in train_loader:
        pred = model(cont_feat)
        loss = criterion(pred, clabel)
        
        # Update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(epoch, loss.item(), amex_metric(clabel.detach().numpy(), pred.detach().numpy()))
    

0 0.783491849899292 0.0763621973857912


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [291]:
 cont_feat.shape 

torch.Size([2064, 177])

In [ ]:
def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)

In [197]:
def amex_metric(y_true, y_pred, return_components=False) -> float:
    """Amex metric for ndarrays"""
    def top_four_percent_captured(df) -> float:
        """Corresponds to the recall for a threshold of 4 %"""
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(df) -> float:
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(df) -> float:
        """Corresponds to 2 * AUC - 1"""
        df2 = pd.DataFrame({'target': df.target, 'prediction': df.target})
        df2.sort_values('prediction', ascending=False, inplace=True)
        return weighted_gini(df) / weighted_gini(df2)

    df = pd.DataFrame({'target': y_true.ravel(), 'prediction': y_pred.ravel()})
    df.sort_values('prediction', ascending=False, inplace=True)
    g = normalized_weighted_gini(df)
    d = top_four_percent_captured(df)

    if return_components: return g, d, 0.5 * (g + d)
    return 0.5 * (g + d)

In [206]:
d = pd.read_csv(data_dir + 'train_data.csv', index_col='customer_ID', usecols=['customer_ID', 'P_2'])
ave_p2 = (d.groupby('customer_ID').mean().rename(columns={'P_2': 'prediction'}))
ave_p2['prediction'] = 1.0 - (ave_p2['prediction'] / ave_p2['prediction'].max())

In [207]:
amex_metric(l, ave_p2['prediction'].values)


0.5729004331080327

True